# Content-based Filtering

Aproach:
- Table mit Features und Ratings erstellen, ohne User - auf Basis des OMDB Datasets

- sklearn-kNN mit cosine-similarity darauf anwenden

- Funktion schreiben, die auf basis des ratings der neighbours das rating eines Filmes vorhersagt.


Sources:

#### https://heartbeat.fritz.ai/recommender-systems-with-python-part-i-content-based-filtering-5df4940bd831

#### https://www.kaggle.com/johnwill225/movie-recommendations

#### https://towardsdatascience.com/how-we-built-a-content-based-filtering-recommender-system-for-music-with-python-c6c3b1020332


In [123]:
import pandas as pd
import numpy as np
import string
import math
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import PCA

from knn_preprocessing import knn_preprocessing

kwargs = dict(random_state=42)

In [2]:
movies = pd.read_csv('../../data/preprocessed/movies_id_updated.csv')
average_ratings = pd.read_csv('../../data/preprocessed/average_ratings.csv')
omdb = pd.read_csv('../../data/preprocessed/omdb_cleaned.csv')
ratings = pd.read_csv('../../data/preprocessed/ratings_clean_std_0.csv')

In [6]:
omdb.columns

Index(['Title', 'Year', 'Rated', 'Runtime', 'Writer', 'Plot', 'Language',
       'imdbRating', 'imdbVotes', 'imdbID', 'Rotten Tomatoes', 'Metacritic',
       'Series', 'Released_season', 'Released_month', 'Released_day',
       'PG_Rating', 'Available_languages', 'Oscars_won', 'Oscars_nominated',
       'Golden_globe_won', 'Golden_globe_nominated'],
      dtype='object')

In [124]:
merged_data = knn_preprocessing(['imdbID', 'Year', 'Runtime', 'Language', 'imdbRating', 'imdbVotes', 'Rotten Tomatoes', 'Metacritic',
       'Series', 'PG_Rating', 'Oscars_won', 'Oscars_nominated',
       'Golden_globe_won', 'Golden_globe_nominated'])

In [125]:
merged_data

,user_id,imdbID,rating,Year,Runtime,imdbRating,imdbVotes,RottenTomatoes,Metacritic,Series,PG_Rating,Oscars_won,Oscars_nominated,Golden_globe_won,Golden_globe_nominated
0,1264,47034.0,3.5,1954.0,96.0,7.6,27485.0,9.3,7.800000,0.0,0.0,0.0,0.0,0.0,0.0
1,213,304141.0,2.5,2004.0,142.0,7.9,524511.0,9.0,8.200000,0.0,1.0,0.0,2.0,0.0,0.0
2,593,369436.0,3.0,2008.0,88.0,5.7,61335.0,2.4,4.100000,0.0,2.0,0.0,0.0,0.0,0.0
3,609,1077258.0,4.0,2007.0,105.0,7.1,196149.0,7.4,5.779918,0.0,0.0,0.0,0.0,0.0,0.0
4,1590,52182.0,4.0,1958.0,100.0,7.4,6337.0,6.9,5.779918,0.0,0.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787536,1032,83530.0,3.0,1982.0,85.0,6.2,47327.0,4.2,4.800000,0.0,1.0,0.0,0.0,0.0,0.0
787537,99,107798.0,3.0,1993.0,141.0,6.6,73544.0,5.3,5.000000,0.0,2.0,0.0,0.0,0.0,0.0
787538,333,93857.0,3.0,1987.0,88.0,5.1,12882.0,0.7,2.800000,0.0,2.0,0.0,0.0,0.0,0.0
787539,49,144168.0,3.0,1999.0,94.0,6.1,17926.0,7.1,6.500000,0.0,2.0,0.0,0.0,0.0,0.0


In [126]:
merged_data.isna().sum()

user_id                   0
imdbID                    0
rating                    0
Year                      0
Runtime                   0
imdbRating                0
imdbVotes                 0
RottenTomatoes            0
Metacritic                0
Series                    0
PG_Rating                 0
Oscars_won                0
Oscars_nominated          0
Golden_globe_won          0
Golden_globe_nominated    0
dtype: int64

In [102]:
# function that computes a rating based on the neighbors
def compute_rating(neighbors, distances):
    # TODO: Function schreiben, die statt dem mean() der Nachbarn die Ratings auf Basis des Abstandes gewichtet
    # Weiter entfernte Filme werden geringer gewichtet nähere Filme werden stärker gewichtet
    
    #Frage an Maxi: Macht er hier automatisch schon das jeweils richtige Rating oder was wird bei neighbors['rating'] gemacht?
    
    #for i in range(0,len(distances[0])):
     
        #pred = (1-(distances[0][i]/distances.sum()))*neighbors[i,3]
    
    pred = np.mean(neighbors)
    
    return float(pred)

In [76]:
ratings = merged_data.loc[merged_data['user_id'] == 123]

In [77]:
real_ratings = ratings.loc[(ratings['imdbID'] == 113627.0)]

In [78]:
real_idx = ratings.loc[(ratings['imdbID'] == 113627.0)].index

In [79]:
print(real_idx)

Int64Index([218118], dtype='int64')


In [80]:
ratings = ratings[ratings['imdbID'] != 113627.0] 

In [81]:
ratings

,index,user_id,imdbID,rating,Year,Runtime,imdbRating,imdbVotes,RottenTomatoes,Metacritic,Series,PG_Rating,Oscars_won,Oscars_nominated,Golden_globe_won,Golden_globe_nominated
2700,2700,123,107688.0,5.0,1993.0,76.0,8.0,282449.0,9.5,8.2,0.0,1.0,0.0,1.0,0.0,0.0
20972,20972,123,319061.0,4.0,2003.0,125.0,8.0,404894.0,7.5,5.8,0.0,2.0,0.0,1.0,0.0,0.0
51933,51933,123,118715.0,5.0,1998.0,117.0,8.1,702389.0,8.2,7.1,0.0,3.0,0.0,0.0,0.0,0.0
69225,69225,123,119822.0,3.5,1997.0,139.0,7.7,267139.0,8.5,6.7,0.0,2.0,2.0,0.0,0.0,0.0
91942,91942,123,338135.0,5.0,2003.0,99.0,7.6,27600.0,8.2,7.0,0.0,3.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742419,742419,123,248845.0,5.0,2001.0,95.0,7.7,30873.0,9.2,8.5,0.0,3.0,0.0,0.0,0.0,1.0
745967,745967,123,112281.0,1.5,1995.0,90.0,6.4,196970.0,3.1,4.5,0.0,2.0,0.0,0.0,0.0,0.0
768099,768099,123,108052.0,5.0,1993.0,195.0,8.9,1165580.0,9.7,9.4,0.0,3.0,7.0,0.0,0.0,0.0
779240,779240,123,181875.0,5.0,2000.0,122.0,7.9,243109.0,8.9,9.0,0.0,3.0,1.0,0.0,0.0,0.0


In [82]:
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=16, n_jobs=-1)

In [83]:
ratings = ratings.drop(columns = {'imdbID','user_id', 'rating','index'})

In [84]:
ratings

,Year,Runtime,imdbRating,imdbVotes,RottenTomatoes,Metacritic,Series,PG_Rating,Oscars_won,Oscars_nominated,Golden_globe_won,Golden_globe_nominated
2700,1993.0,76.0,8.0,282449.0,9.5,8.2,0.0,1.0,0.0,1.0,0.0,0.0
20972,2003.0,125.0,8.0,404894.0,7.5,5.8,0.0,2.0,0.0,1.0,0.0,0.0
51933,1998.0,117.0,8.1,702389.0,8.2,7.1,0.0,3.0,0.0,0.0,0.0,0.0
69225,1997.0,139.0,7.7,267139.0,8.5,6.7,0.0,2.0,2.0,0.0,0.0,0.0
91942,2003.0,99.0,7.6,27600.0,8.2,7.0,0.0,3.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
742419,2001.0,95.0,7.7,30873.0,9.2,8.5,0.0,3.0,0.0,0.0,0.0,1.0
745967,1995.0,90.0,6.4,196970.0,3.1,4.5,0.0,2.0,0.0,0.0,0.0,0.0
768099,1993.0,195.0,8.9,1165580.0,9.7,9.4,0.0,3.0,7.0,0.0,0.0,0.0
779240,2000.0,122.0,7.9,243109.0,8.9,9.0,0.0,3.0,1.0,0.0,0.0,0.0


In [263]:
pd.DataFrame(ratings.index)

,0
0,2700
1,20972
2,51933
3,69225
4,91942
...,...
62,742419
63,745967
64,768099
65,779240


In [85]:
scaler = preprocessing.StandardScaler()
features = pd.DataFrame(scaler.fit_transform(ratings)).merge(pd.DataFrame(ratings.index), left_index=True, right_index=True, how='left')

In [86]:
features

,0_x,1,2,3,4,5,6,7,8,9,10,11,0_y
0,0.101151,-1.541061,0.646568,-0.007216,0.906616,0.775941,0.0,-0.982093,-0.463768,0.224904,0.0,-0.203595,2700
1,0.772155,0.148753,0.646568,0.333172,-0.076285,-0.616902,0.0,-0.104757,-0.463768,0.224904,0.0,-0.203595,20972
2,0.436653,-0.127135,0.764606,1.160186,0.267730,0.137555,0.0,0.772580,-0.463768,-0.528524,0.0,-0.203595,51933
3,0.369553,0.631557,0.292453,-0.049777,0.415166,-0.094586,0.0,-0.104757,0.731327,-0.528524,0.0,-0.203595,69225
4,0.772155,-0.747883,0.174415,-0.715678,0.267730,0.079520,0.0,0.772580,0.133779,-0.528524,0.0,-0.203595,91942
...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,0.637954,-0.885827,0.292453,-0.706579,0.759181,0.950047,0.0,0.772580,-0.463768,-0.528524,0.0,3.206616,742419
63,0.235352,-1.058257,-1.242044,-0.244842,-2.238667,-1.371359,0.0,-0.104757,-0.463768,-0.528524,0.0,-0.203595,745967
64,0.101151,2.562773,1.708912,2.447824,1.004906,1.472363,0.0,0.772580,3.719066,-0.528524,0.0,-0.203595,768099
65,0.570854,0.045295,0.528530,-0.116578,0.611746,1.240223,0.0,0.772580,0.133779,-0.528524,0.0,-0.203595,779240


In [87]:
features.iloc[:,0:12]

,0_x,1,2,3,4,5,6,7,8,9,10,11
0,0.101151,-1.541061,0.646568,-0.007216,0.906616,0.775941,0.0,-0.982093,-0.463768,0.224904,0.0,-0.203595
1,0.772155,0.148753,0.646568,0.333172,-0.076285,-0.616902,0.0,-0.104757,-0.463768,0.224904,0.0,-0.203595
2,0.436653,-0.127135,0.764606,1.160186,0.267730,0.137555,0.0,0.772580,-0.463768,-0.528524,0.0,-0.203595
3,0.369553,0.631557,0.292453,-0.049777,0.415166,-0.094586,0.0,-0.104757,0.731327,-0.528524,0.0,-0.203595
4,0.772155,-0.747883,0.174415,-0.715678,0.267730,0.079520,0.0,0.772580,0.133779,-0.528524,0.0,-0.203595
...,...,...,...,...,...,...,...,...,...,...,...,...
62,0.637954,-0.885827,0.292453,-0.706579,0.759181,0.950047,0.0,0.772580,-0.463768,-0.528524,0.0,3.206616
63,0.235352,-1.058257,-1.242044,-0.244842,-2.238667,-1.371359,0.0,-0.104757,-0.463768,-0.528524,0.0,-0.203595
64,0.101151,2.562773,1.708912,2.447824,1.004906,1.472363,0.0,0.772580,3.719066,-0.528524,0.0,-0.203595
65,0.570854,0.045295,0.528530,-0.116578,0.611746,1.240223,0.0,0.772580,0.133779,-0.528524,0.0,-0.203595


In [90]:
knn.fit(features.iloc[:,0:12])

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=16)

In [88]:
input_data = merged_data.iloc[real_idx]

In [91]:
inputs = scaler.transform(input_data.drop(columns = {'imdbID','user_id', 'rating','index'}))

In [92]:
distances , indices = knn.kneighbors(inputs, n_neighbors=16)

In [93]:
distances

array([[0.04011158, 0.05912441, 0.12696375, 0.13029713, 0.13450754,
        0.22758743, 0.2338377 , 0.27206453, 0.54052198, 0.54282328,
        0.62628369, 0.62823152, 0.65810453, 0.69028813, 0.70046945,
        0.72784601]])

In [94]:
indices

array([[34, 27, 65, 16, 53,  4, 47, 15, 19, 45, 62, 25, 13,  6,  9, 18]])

In [99]:
features['0_y'].loc[indices[0]]

34    395310
27    311564
65    779240
16    242684
53    607882
4      91942
47    563897
15    239963
19    265072
45    488721
62    742419
25    300627
13    195926
6      93789
9     140307
18    253199
Name: 0_y, dtype: int64

In [101]:
merged_data['rating'].loc[features['0_y'].loc[indices[0]]]

395310    5.0
311564    5.0
779240    5.0
242684    4.5
607882    5.0
91942     5.0
563897    5.0
239963    0.5
265072    5.0
488721    2.5
742419    5.0
300627    3.5
195926    0.5
93789     5.0
140307    5.0
253199    5.0
Name: rating, dtype: float64

In [111]:
## function that predicts the rating of a movie from its imdbID and its 20 nearest neighbors

def predict_movie_rating(imdbID, userID, user_data):
    # get 20 (or x) nearest neighbors that were rated by userID
    # TODO: Use function from kNN.ipynb to find optimal k for algorithm 
    k_neighbors = 16    
    
    # Select all ratings given by User #userID
    ratings = user_data.loc[user_data['user_id'] == userID]
    
    # Get real rating -> remove this in the end -> currently done for validation
    real_ratings = ratings.loc[(ratings['imdbID'] == imdbID)]
    
    real_idx = ratings.loc[(ratings['imdbID'] == imdbID)].index
    
    #remove real rating
    ratings = ratings[ratings['imdbID'] != imdbID] 
    
    ratings = ratings.drop(columns = {'imdbID','user_id', 'rating','index'})
    
    scaler = preprocessing.StandardScaler()
    features = pd.DataFrame(scaler.fit_transform(ratings)).merge(pd.DataFrame(ratings.index), left_index=True, right_index=True, how='left')
    
    
    
    if (ratings.to_numpy().size>0):   
        
        # Convert to Sparse Matrix Format
        #ratings_as_csr = csr_matrix(ratings)
        
        # Set algorithm and params 
        # TODO adapt neighbors based on number of given ratings by user_id ?
        knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=k_neighbors, n_jobs=-1)

        # Training
        #print('---- Training ConBF-kNN-Algorithm ----')
        #print('user_id: '+str(userID))
        #print('imdbID: '+str(imdbID))
        
        knn.fit(features.iloc[:,0:12])
        
        input_data = user_data.iloc[real_idx]
        inputs = scaler.transform(input_data.drop(columns = {'imdbID','user_id', 'rating','index'}))
        
        
        #Prediction -> get x nearest neighbors of imdbID
        distances , indices = knn.kneighbors(inputs, n_neighbors=k_neighbors)
        
       
        neighbor_ratings = user_data['rating'].loc[features['0_y'].loc[indices[0]]]
        
        # compute rating of movie(imbdID) based on the rating of the 20 nearest neighbors
        pred = compute_rating(neighbor_ratings, distances)
        
        #Generate Output for Understandability
        #print('Predicted Rating for '+str(imdbID)+': '+str(pred))
        #print('Real Rating of '+str(imdbID)+' was: '+ str(real_ratings['rating'].values[0]))
        
        # return rating prediction and real rating
        return pred , real_ratings['rating'].values[0]
        
    else:
         return "User has not rated other movies. Check input"
    

In [104]:
### Testing function ###
imdbID = 113627.0
# Aufpassen userID und imdbID als float übergeben!!
userID = 123
pred , real = predict_movie_rating(imdbID, userID, merged_data)

Predicted Rating for 113627.0: 4.15625
Real Rating of 113627.0 was: 5.0


In [105]:
### Testing function ###
imdbID = 145653
# Aufpassen userID und imdbID als float übergeben!!
userID = 123
pred , real = predict_movie_rating(imdbID, userID, merged_data)

Predicted Rating for 145653: 2.875
Real Rating of 145653 was: 4.5


In [63]:
## DUPLICATED RATING!!! -> durch das umschreiben mit .values[0] wird das Doppelte Rating nicht mehr angezeigt -> tdm mal checken!

In [106]:
# Larger Test:
# TODO Refactoring needed #UglyPythonCode

def test_predict_mr(no_test_samples):
    # Sampling #no_test_samples of random samples from dataset for testing
    test_set = merged_data.sample(n=no_test_samples)
    
    predictions = pd.DataFrame(columns=['Prediction'])
    reals = pd.DataFrame(columns=['Real_Rating'])
    
    # Iterate over test-set and generate predicitons for it
    # TODO get rid of ugly for-loop
    for row in test_set.itertuples():
        imdbID = row.imdbID
        userID = row.user_id
        pred , real = predict_movie_rating(imdbID, userID, merged_data)
        predictions.loc[row.index] = pred
        reals.loc[row.index] = real
    
    rmse = mean_squared_error(reals['Real_Rating'], predictions['Prediction'], squared=False)
    print('RMSE: '+str(rmse))
    return float(rmse)

In [ ]:
n = 50
test_predict_mr(n)

In [140]:
upper = 0
lower = 10

for i in range(0,10):
    n = 100
    rmse = test_predict_mr(n)
    
    if (float(rmse) < float(lower)):
        lower = rmse
    
    if (float(rmse) > float(upper)):
        upper = rmse
        
print('RMSE upper Bound: '+str(upper))
print('RMSE lower Bound: '+str(lower))

RMSE: 1.2051124004709146
RMSE: 1.3530600957644505
RMSE: 1.3287519619612989
RMSE: 1.2780180751575112
RMSE: 1.2437837398829847
RMSE: 1.1412267946764523
RMSE: 1.3845269303683074
RMSE: 1.1319490041349227
RMSE: 1.3490357591740214
RMSE: 1.171923103627717
RMSE upper Bound: 1.3845269303683074
RMSE lower Bound: 1.1319490041349227


In [ ]:
n = 1000
test_predict_mr(n)

In [ ]:
#Tested with (metric='minkowski', p=2)
n = 50
test_predict_mr(n)

In [150]:
#Tested with (metric='minkowski', p=2)
n = 50
test_predict_mr(n)

RMSE: 1.0720381692131251


1.0720381692131251

In [113]:
#Tested with minkowski
upper = 0
lower = 10

for i in range(0,10):
    n = 10000
    rmse = test_predict_mr(n)
    
    if (float(rmse) < float(lower)):
        lower = rmse
    
    if (float(rmse) > float(upper)):
        upper = rmse
        
print('RMSE upper Bound: '+str(upper))
print('RMSE lower Bound: '+str(lower))

RMSE: 0.8058140172947788
RMSE: 0.8175548887345424
RMSE: 0.8179095161217407
RMSE: 0.8080798638787196
RMSE: 0.8129051273634273
RMSE: 0.8155967428055362
RMSE: 0.8115931838635352
RMSE: 0.8270465265214829
RMSE: 0.8208911649466694
RMSE: 0.8115055813694383
RMSE upper Bound: 0.8270465265214829
RMSE lower Bound: 0.8058140172947788
